In [63]:
from google import genai
from google.genai import types
import os,re
import pathlib
import httpx
import pandas as pd
from io import StringIO
from dotenv import load_dotenv
load_dotenv()

True

In [64]:
# Retrieve and encode the PDF byte
f_path= "/ephemeral/home/xiong/data/Fund/RAM_Table/RAM_Table_Examples/122_2022_0.pdf"
filepath = pathlib.Path(f_path)

In [ ]:
client = genai.Client(api_key=os.getenv("GEMINI_API_KEY"))

In [ ]:
PROMPT_RAM = """
**Objective:** Extract data from Risk Assessment Matrix (RAM) tables within a provided document and output it into a structured CSV file.

**Phase 1: Identification of Relevant Risk Assessment Matrix Tables**
Carefully examine the document to identify the correct table(s) based on the following ordered criteria:

1.  **Primary Keyword Search:**
    * Look for titles or headings immediately preceding a table that contain any of these exact phrases (case-insensitive): "RAM", "Risk Assessment Matrix", "Overall Level of Concern".
    * Look for these same phrases within the table itself, either as a table heading or as a column header.
2.  **Handling Multi-Page Tables:**
    * If a table matching the criteria spans multiple pages, ensure all segments of the table are consolidated and treated as a single entity.
3.  **Appendix Search (Secondary Location):**
    * If no table is identified in the main body of the document using the primary keywords, search within any section labeled as "Appendix" (or similar, e.g., "Annex").
    * In the appendix, look for tables that contain column names such as (or similar to): "Source", "Risk", "Threat", "Likelihood", AND "Impact". The presence of columns related to both likelihood and impact is crucial here.
4.  **No Table Found:**
    * If, after following all the above steps, no suitable table can be confidently identified, return nothing or an empty CSV.

**Phase 2: Data Extraction and CSV Output Formatting**
Once a relevant table is identified, extract its data and structure it into a CSV file with the following precise columns. Apply these rules to each row of the identified table(s):

1.  **`Heading`**:
    * Content: Store the table's primary title or caption if available. If multiple (e.g., a number and a title), combine them. If no explicit title/caption is found directly associated with the table, leave this field empty for that table's rows.
2.  **`Risk Type`**:
    * Source: Identify the row, column or section headers  in the source table that most likely contains types of risks. They oftern appear as 'Global Risks','External Risks' or 'Domestic Risks','Country Specific Risks', or similar.
    * Content: it should be classificed as either "Global Risks" Or "Domestic Risks". This field cannot be empty. 
3.  **`Risk`**:
    * Source: Identify the column in the source table that most likely contains risk descriptions. Look for column headers containing keywords like "Risk", "Threat", "Hazard", or "Source".
    * Content: Extract the full text from this cell. This field cannot be empty. It often contains long, descriptive text.
4.  **`Likelihood Full`**:
    * Source: Identify the column in the source table that most likely contains detailed likelihood information. Look for column headers containing the keyword "Likelihood".
    * Content: Extract EVERYTHING from this cell, including multiple paragraphs, bullet points, and any text separated by large blank spaces. This field cannot be empty.
5.  **`Likelihood`**:
    * Source: From the content extracted for `Likelihood Full`.
    * Content: Extract only the first line or primary categorical value that typically describes the level of likelihood (e.g., "High", "Medium", "Low", "Likely", "Unlikely", "Remote", "Possible", "Frequent"). If the first line is a detailed sentence, try to infer the single-word or short-phrase category.
6.  **`Impact Full`**:
    * Source: Identify the column in the source table that most likely contains detailed impact information. Look for column headers containing the keyword "Impact" or "Consequence".
    * Content: Extract EVERYTHING from this cell, including multiple paragraphs, bullet points, and any text separated by large blank spaces. This field cannot be empty.
7.  **`Impact`**:
    * Source: From the content extracted for `Impact Full`.
    * Content: Extract only the first line or primary categorical value that typically describes the level of impact (e.g., "High", "Medium", "Low", "Severe", "Moderate", "Minor", "Catastrophic", "Significant"). If the first line is a detailed sentence, try to infer the single-word or short-phrase category.
8.  **`Policy Response`**:
    * Source: Identify the column in the source table that most likely contains information about policy responses to the risk. Look for column headers containing keywords like "Policy", "Response", "Mitigation", "Action", or "Measure".
    * Content: Extract the full text from this cell. If no such column is found, or if the cell for a given risk is empty, leave this field empty in the CSV.

**Phase 3: Important Processing Details**

1.  **Footnote Removal:** If footnotes or source notes are present directly beneath the table content (and are clearly distinguishable from the main table data), they should be excluded from the extraction.
2.  **Data Integrity:**
    * If a row within an identified RAM table is missing data for the determined "Risk", "Likelihood Full", or "Impact Full" source columns, still include the row but leave the corresponding CSV cells empty for that missing piece (this slightly overrides "can't be empty" for cases where a source cell *within an otherwise valid RAM row* is blank). The expectation is that most rows in a valid RAM will have this data.
    * Prioritize accurate mapping of table columns to the requested CSV columns.

**Output:**
Provide the extracted data as a single CSV file.

"""

In [ ]:
def extract_csv_content(text):
    match = re.search(r"```csv(.*?)```", text, re.DOTALL)
    if match:
        return match.group(1).strip()
    return text.strip()

def clean_markdown_fence(text):
    """
    Removes Markdown code fences like ```csv ... ``` and returns only the inner content.
    Handles cases with or without newlines, with or without language tags.
    """
    # Match fenced code block with or without language tag
    match = re.search(r"```(?:\w+)?(.*?)```", text, re.DOTALL)
    if match:
        return match.group(1).strip()
    return text.strip()

def clean_RAM(ram):

    letter_pattern = r'[a-zA-Z]'
    keyword_pattern = r'(risk|assessment|matrix|ram)'
    likelihood_impact_keywords = ['high', 'medium', 'low', 'moderate', 'severe']

    # Remove rows where 'Likelihood' or 'Impact' columns contain no letters

    cleaned_df = ram[
        ram['Likelihood'].str.contains(letter_pattern, na=False) &
        ram['Impact'].str.contains(letter_pattern, na=False) &
        ram['Likelihood Full'].str.contains(letter_pattern, na=False) &
        ram['Impact Full'].str.contains(letter_pattern, na=False)
    ]

    # Remove rows where 'Heading' does not contain RAM keywords if 'Heading' is not missing
    cleaned_df = cleaned_df[
        cleaned_df['Heading'].isna() | 
        (cleaned_df['Heading'].str.count(keyword_pattern, flags=re.IGNORECASE) >=2)
    ]

    # Remove rows where 'Likelihood' or 'Impact' columns do not contain keywords from high, medium, low
    cleaned_df = cleaned_df[
        cleaned_df['Likelihood'].str.contains('|'.join(likelihood_impact_keywords), case=False, na=False) |
        cleaned_df['Impact'].str.contains('|'.join(likelihood_impact_keywords), case=False, na=False)
    ]

    # Keep only the specified columns
    cleaned_df = cleaned_df[['Heading', 'Risk Type','Risk', 'Likelihood', 'Likelihood Full', 'Impact', 'Impact Full','Policy Response']]

    print(f"Rows before cleaning: {len(ram)}")
    print(f"Rows after cleaning: {len(cleaned_df)}")

    return cleaned_df

- process to convert to csv directly 

In [91]:
response = client.models.generate_content(
          model="gemini-2.5-pro-preview-05-06",
          contents=[
              types.Part.from_bytes(
                data=filepath.read_bytes(),
                mime_type='application/pdf',
              ),
              PROMPT_RAM])


In [92]:
print(response.text)

```csv
Heading,Risk Type,Risk,Likelihood Full,Likelihood,Impact Full,Impact,Policy Response
"Annex IV. Risk Assessment Matrix","Global Risks","Russia's invasion of Ukraine leads to escalation of sanctions and other disruptions. Russia is disconnected almost completely from the global financial system and large parts of the trading system. This, combined with Russian countersanctions and secondary sanctions on countries and companies that continue business with Russia, leads to even higher commodity prices, refugee migration, tighter financial conditions.","High","High","High
• Lower consumption and production, particular manufacturing industry due to gas supply disruption.
• Lower exports due to weakened external demand.
• Lower investment due to heightened uncertainty and tightening of financial conditions.
• Increase systemic risks in the financial sector.","High","Accelerate green transitions
Diversify energy sources
Provide targeted fiscal support to vulnerable households.
Careful 

In [93]:
df = pd.read_csv(StringIO(extract_csv_content(response.text)))
if df.empty:
    print("No relevant RAM table found.")
else:
    df = clean_RAM(df)

Rows before cleaning: 9
Rows after cleaning: 9


In [94]:
df

,Heading,Risk Type,Risk,Likelihood Full,Likelihood,Impact Full,Impact,Policy Response
0,Annex IV. Risk Assessment Matrix,Global Risks,Russia's invasion of Ukraine leads to escalati...,High,High,"High\n• Lower consumption and production, part...",High,Accelerate green transitions\nDiversify energy...
1,Annex IV. Risk Assessment Matrix,Global Risks,Rising and volatile food and energy prices. Co...,High,High,Medium\n• Lower growth due to weak consumption...,Medium,Provide targeted fiscal support to vulnerable ...
2,Annex IV. Risk Assessment Matrix,Global Risks,Outbreaks of lethal and highly contagious COVI...,High,High,"Medium\n• Tourist arrivals remain low, lowerin...",Medium,Targeted support to direct- affected businesse...
3,Annex IV. Risk Assessment Matrix,Global Risks,De-anchoring of inflation expectations in the ...,Medium for the US and Medium- low for the Euro...,Medium,Medium\n• Weaker confidence could reduce inves...,Medium,• Implement reforms to boost productivity and ...
4,Annex IV. Risk Assessment Matrix,Global Risks,Geopolitical tensions and de- globalization. I...,High,High,High\n• Tariff and non-tariff restrictions fur...,High,• Allow automatic stabilizer fully to operate....
5,Annex IV. Risk Assessment Matrix,Global Risks,Higher frequency and severity of natural disas...,Medium,Medium,Medium\n• Weaker demand for winter tourism and...,Medium,• Support affected sectors and rebuild damaged...
6,Annex IV. Risk Assessment Matrix,Domestic Risks,Extended global supply chain disruptions. Pers...,High,High,Medium/High\n• Persistently high inflation\n• ...,Medium/High,Provide targeted fiscal support to vulnerable ...
7,Annex IV. Risk Assessment Matrix,Domestic Risks,Weakness in coordinating and implementing dome...,Medium,Medium,Medium\n• Reform setback could undermine confi...,Medium,Outline reform details in a comprehensive pack...
8,Annex IV. Risk Assessment Matrix,Domestic Risks,Contagion from CESEE. The stock of foreign cur...,High,High,Medium\n• Depreciation in host country could r...,Medium,Boost structural profitability through reducin...


- process to convert to markdown 

In [77]:
from IPython.display import Markdown, display

In [73]:
PROMPT_RAM_MD = """
**Objective:** Convert data from Risk Assessment Matrix (RAM) tables within a provided document and output it into a Markdown.

**Phase 1: Identification of Relevant Risk Assessment Matrix Tables**
Carefully examine the document to identify the correct table(s) based on the following ordered criteria:

1.  **Primary Keyword Search:**
    * Look for titles or headings immediately preceding a table that contain any of these exact phrases (case-insensitive): "RAM", "Risk Assessment Matrix", "Overall Level of Concern".
    * Look for these same phrases within the table itself, either as a table heading or as a column header.
2.  **Handling Multi-Page Tables:**
    * If a table matching the criteria spans multiple pages, ensure all segments of the table are consolidated and treated as a single entity.
3.  **Appendix Search (Secondary Location):**
    * If no table is identified in the main body of the document using the primary keywords, search within any section labeled as "Appendix" (or similar, e.g., "Annex").
    * In the appendix, look for tables that contain column names such as (or similar to): "Source", "Risk", "Threat", "Likelihood", AND "Impact". The presence of columns related to both likelihood and impact is crucial here.
4.  **No Table Found:**
    * If, after following all the above steps, no suitable table can be confidently identified, return nothing.
    
**Phase 2: Data Extraction and Markdown Output Formatting**
1.  Once a relevant table is identified, extract its data and structure it into a Markdown format.

**Phase 3: Important Processing Details**
1.  **Footnote Removal:** If footnotes or source notes are present directly beneath the table content (and are clearly distinguishable from the main table data), they should be excluded from the extraction.
2.  **Data Integrity:** the output content should be the exact content from the table, without any paraphrasing, summarization or modification in wording. 

**Output:**
Provide the extracted data as a Markdown file.
"""

In [74]:
response = client.models.generate_content(
          model="gemini-2.5-pro-preview-05-06",
          contents=[
              types.Part.from_bytes(
                data=filepath.read_bytes(),
                mime_type='application/pdf',
              ),
              PROMPT_RAM_MD])

In [76]:
print(response.text)

```markdown
| Risks | Relative Likelihood | Impact if Realized | Policy Response |
|---|---|---|---|
| **External Risks** |  |  |  |
| Russia's invasion of Ukraine leads to escalation of sanctions and other disruptions. Russia is disconnected almost completely from the global financial system and large parts of the trading system. This, combined with Russian countersanctions and secondary sanctions on countries and companies that continue business with Russia, leads to even higher commodity prices, refugee migration, tighter financial conditions. | High | High | Accelerate green transitions<br>Diversify energy sources<br>Provide targeted fiscal support to vulnerable households.<br>Careful monitoring of financial sector developments and increase buffers.<br>Allow automatic stabilizer fully to operate. |
| Rising and volatile food and energy prices. Commodity prices are volatile and trend up amid pent-up demand and supply disruptions, conflicts, or a bumpy transition to renewable energy 

In [80]:
display(Markdown(clean_markdown_fence(response.text)))

| Risks | Relative Likelihood | Impact if Realized | Policy Response |
|---|---|---|---|
| **External Risks** |  |  |  |
| Russia's invasion of Ukraine leads to escalation of sanctions and other disruptions. Russia is disconnected almost completely from the global financial system and large parts of the trading system. This, combined with Russian countersanctions and secondary sanctions on countries and companies that continue business with Russia, leads to even higher commodity prices, refugee migration, tighter financial conditions. | High | High | Accelerate green transitions<br>Diversify energy sources<br>Provide targeted fiscal support to vulnerable households.<br>Careful monitoring of financial sector developments and increase buffers.<br>Allow automatic stabilizer fully to operate. |
| Rising and volatile food and energy prices. Commodity prices are volatile and trend up amid pent-up demand and supply disruptions, conflicts, or a bumpy transition to renewable energy sources. This leads to bouts of price and real sector volatility, including acute energy crises in some countries. | High | Medium | Provide targeted fiscal support to vulnerable households or heavily affected companies.<br>Ensure adequate compensation to carbon-tax affected households and firms |
| Outbreaks of lethal and highly contagious COVID-19 variants: Rapidly increasing hospitalizations and deaths, due to low vaccination rates or caused by vaccine-resistant variants, force lockdowns and increased uncertainty about the course of the pandemic. | High | Medium | Targeted support to direct-affected businesses and households.<br>Maintain public health safeguard measures, including contact tracing and widespread testing.<br>Use available fiscal space to support households and firms and encourage reallocation of resources. |
| De-anchoring of inflation expectations in the U.S. and/or advanced European economies. A fast recovery in demand amid a lagging supply-side response leads to a rapid de-anchoring of inflation expectations, which prompts central banks to tighten policies abruptly. | Medium for the US and Medium-low for the Euro Area | Medium | • Weaker confidence could reduce investment.<br>• Persistently high inflation and lower purchasing power. | • Implement reforms to boost productivity and potential growth. |
| Geopolitical tensions and de-globalization. Intensified geopolitical tensions, security risks, and conflicts cause economic and political disruptions, disorderly migration, production reshoring, a decline in global trade, and lower investor confidence. Associated supply chain disruptions and commodity price shocks give rise to inflationary pressures. | High | High | • Tariff and non-tariff restrictions further dampen Austria's export and growth prospects | • Allow automatic stabilizer fully to operate.<br>• Participate in global initiatives supporting multilateralism |
| Higher frequency and severity of natural disasters related to climate change cause severe economic damage and prompt a recalculation of risk and growth prospects. | Medium | Medium | • Weaker demand for winter tourism and lower growth. | • Support affected sectors and rebuild damaged infrastructure.<br>• Reallocate fiscal spending as needed.<br>• Diversifying tourism export base. |
| **Domestic Risks** |  |  |  |
| Extended global supply chain disruptions. Persistent disruptions in the production and shipment of components caused by lockdowns and logistical bottlenecks continue until 2023. | High | Medium/ High | • Persistently high inflation<br>• Lower purchasing power<br>• Weaker investment | Provide targeted fiscal support to vulnerable households or heavily affected companies.<br>In the event of a permanent shock affecting potential growth, allow smooth reallocation of resources through restructuring and insolvency procedures, and ALMPs. |
| Weakness in coordinating and implementing domestic reforms. The government has an ambitious reform agenda that includes large-scale investment, institutional cost-efficiency savings, and green reform. | Medium | Medium | • Reform setback could undermine confidence | Outline reform details in a comprehensive package.<br>Identify key measures on the expenditure side.<br>Publish progress reports periodically. |
| Contagion from CESEE. The stock of foreign currency loans in CESEE is still high and Austrian banks are susceptible to host currency depreciation. | High | Medium | • Depreciation in host country could raise debt burdens of borrowers and increase NPLs of Austrian banks. | Boost structural profitability through reducing operating cost. |

### Experiment with full document without specific Table Identification 

In [95]:
full_doc_path= '/ephemeral/home/xiong/data/Fund/pdf_parse/Fund_Document/input/All_AIV_2008-2025_PDF/122_2022_0.pdf'
filepath = pathlib.Path(f_path)
filepath.exists()

True

In [96]:
response = client.models.generate_content(
          model="gemini-2.5-pro-preview-05-06",
          contents=[
              types.Part.from_bytes(
                data=filepath.read_bytes(),
                mime_type='application/pdf',
              ),
              PROMPT_RAM])

In [97]:
df = pd.read_csv(StringIO(extract_csv_content(response.text)))
if df.empty:
    print("No relevant RAM table found.")
else:
    df = clean_RAM(df)

Rows before cleaning: 9
Rows after cleaning: 9


In [98]:
df

,Heading,Risk Type,Risk,Likelihood Full,Likelihood,Impact Full,Impact,Policy Response
0,Annex IV. Risk Assessment Matrix¹,Global Risks,Russia's invasion of Ukraine leads to escalati...,High,High,"High\n• Lower consumption and production, part...",High,Accelerate green transitions\nDiversify energy...
1,Annex IV. Risk Assessment Matrix¹,Global Risks,Rising and volatile food and energy prices. Co...,High,High,Medium\n• Lower growth due to weak consumption...,Medium,Provide targeted fiscal support to vulnerable ...
2,Annex IV. Risk Assessment Matrix¹,Global Risks,Outbreaks of lethal and highly contagious COVI...,High,High,"Medium\n• Tourist arrivals remain low, lowerin...",Medium,Targeted support to direct-affected businesses...
3,Annex IV. Risk Assessment Matrix¹,Global Risks,De-anchoring of inflation expectations in the ...,Medium\nfor the US\nand\nMedium-\nlow for the\...,Medium,Medium\n• Weaker confidence could reduce inves...,Medium,• Implement reforms to boost productivity and ...
4,Annex IV. Risk Assessment Matrix¹,Global Risks,Geopolitical tensions and de-globalization. In...,High,High,High\n• Tariff and non-tariff restrictions fur...,High,• Allow automatic stabilizer fully to operate....
5,Annex IV. Risk Assessment Matrix¹,Global Risks,Higher frequency and severity of natural disas...,Medium,Medium,Medium\n• Weaker demand for winter tourism and...,Medium,• Support affected sectors and rebuild damaged...
6,Annex IV. Risk Assessment Matrix¹,Domestic Risks,Extended global supply chain disruptions. Pers...,High,High,Medium/High\n• Persistently high inflation\n• ...,Medium/High,Provide targeted fiscal support to vulnerable ...
7,Annex IV. Risk Assessment Matrix¹,Domestic Risks,Weakness in coordinating and implementing dome...,Medium,Medium,Medium\n• Reform setback could undermine confi...,Medium,Outline reform details in a comprehensive pack...
8,Annex IV. Risk Assessment Matrix¹,Domestic Risks,Contagion from CESEE. The stock of foreign cur...,High,High,Medium\n• Depreciation in host country could r...,Medium,Boost structural profitability through reducin...


In [88]:
response = client.models.generate_content(
          model="gemini-2.5-pro-preview-05-06",
          contents=[
              types.Part.from_bytes(
                data=filepath.read_bytes(),
                mime_type='application/pdf',
              ),
              PROMPT_RAM_MD])
display(Markdown(clean_markdown_fence(response.text)))

# Annex IV. Risk Assessment Matrix

**AUSTRIA**

| Risks | Relative Likelihood | Impact if Realized | Policy Response |
|---|---|---|---|
| **External Risks** |  |  |  |
| Russia's invasion of Ukraine leads to escalation of sanctions and other disruptions. Russia is disconnected almost completely from the global financial system and large parts of the trading system. This, combined with Russian countersanctions and secondary sanctions on countries and companies that continue business with Russia, leads to even higher commodity prices, refugee migration, tighter financial conditions. | High | **High** <br> • Lower consumption and production, particular manufacturing industry due to gas supply disruption. <br> • Lower exports due to weakened external demand. <br> • Lower investment due to heightened uncertainty and tightening of financial conditions. <br> • Increase systemic risks in the financial sector. | Accelerate green transitions <br> Diversify energy sources <br> Provide targeted fiscal support to vulnerable households. <br> Careful monitoring of financial sector developments and increase buffers. <br> Allow automatic stabilizer fully to operate. |
| Rising and volatile food and energy prices. Commodity prices are volatile and trend up amid pent-up demand and supply disruptions, conflicts, or a bumpy transition to renewable energy sources. This leads to bouts of price and real sector volatility, including acute energy crises in some countries. | High | **Medium** <br> • Lower growth due to weak consumption and investment <br> • Persistently high inflation and pressure for the second-round effect <br> • Rising social tension especially if linked to uncompensated carbon pricing | Provide targeted fiscal support to vulnerable households or heavily affected companies. <br> Ensure adequate compensation to carbon-tax affected households and firms |
| Outbreaks of lethal and highly contagious COVID-19 variants: Rapidly increasing hospitalizations and deaths, due to low vaccination rates or caused by vaccine-resistant variants, force lockdowns and increased uncertainty about the course of the pandemic. | High | **Medium** <br> • Tourist arrivals remain low, lowering the current account. <br> • Domestic demand especially in contact-intensive sectors will weaken. <br> • Larger economic scarring | Targeted support to direct-affected businesses and households. <br> Maintain public health safeguard measures, including contact tracing and widespread testing. <br> Use available fiscal space to support households and firms and encourage reallocation of resources. |
| De-anchoring of inflation expectations in the U.S. and/or advanced European economies. A fast recovery in demand amid a lagging supply-side response leads to a rapid de-anchoring of inflation expectations, which prompts central banks to tighten policies abruptly. | Medium <br> for the US <br> and <br> Medium- <br> low for the <br> Euro Area | **Medium** <br> • Weaker confidence could reduce investment. <br> • Persistently high inflation and lower purchasing power. | • Implement reforms to boost productivity and potential growth. |
| Geopolitical tensions and de-globalization. Intensified geopolitical tensions, security risks, and conflicts cause economic and political disruptions, disorderly migration, production reshoring, a decline in global trade, and lower investor confidence. Associated supply chain disruptions and commodity price shocks give rise to inflationary pressures. | High | **High** <br> • Tariff and non-tariff restrictions further dampen Austria's export and growth prospects | • Allow automatic stabilizer fully to operate. <br> • Participate in global initiatives supporting multilateralism |
| Higher frequency and severity of natural disasters related to climate change cause severe economic damage and prompt a recalculation of risk and growth prospects. | Medium | **Medium** <br> • Weaker demand for winter tourism and lower growth. | • Support affected sectors and rebuild damaged infrastructure. <br> • Reallocate fiscal spending as needed. <br> • Diversifying tourism export base. |
| **Domestic Risks** |  |  |  |
| Extended global supply chain disruptions. Persistent disruptions in the production and shipment of components caused by lockdowns and logistical bottlenecks continue until 2023. | High | **Medium/High** <br> • Persistently high inflation <br> • Lower purchasing power <br> • Weaker investment | Provide targeted fiscal support to vulnerable households or heavily affected companies. <br> In the event of a permanent shock affecting potential growth, allow smooth reallocation of resources through restructuring and insolvency procedures, and ALMPs. |
| Weakness in coordinating and implementing domestic reforms. The government has an ambitious reform agenda that includes large-scale investment, institutional cost-efficiency savings, and green reform. | Medium | **Medium** <br> • Reform setback could undermine confidence | Outline reform details in a comprehensive package. <br> Identify key measures on the expenditure side. <br> Publish progress reports periodically. |
| Contagion from CESEE. The stock of foreign currency loans in CESEE is still high and Austrian banks are susceptible to host currency depreciation. | High | **Medium** <br> • Depreciation in host country could raise debt burdens of borrowers and increase NPLs of Austrian banks. | Boost structural profitability through reducing operating cost. |